In [1]:
from tulip import tlp
from geometric import *
import math

LOG_BASE = 2
ALPHA = 0.2

In [2]:
# penalty function of overlap area
# m is the maximum possible overlap
# Linear, slope = 1
def penaltyFunc(x, m):
    if x == 0:
        return 0
    elif x < (1 - ALPHA) * m:
        return x + ALPHA * m
    else:
        return m

In [3]:
# Lienar, slope < 1
# def penaltyFunc(x, m):
#     if x == 0:
#         return 0
#     else:
#         return (1 - ALPHA) * x + ALPHA * m

In [4]:
# Log
# def penaltyFunc(x, m):
#     if x == 0:
#         return 0
#     else:
#         return math.log(x + 1, LOG_BASE) + ALPHA * m

In [5]:
# Penalty function for edge edge crossing
# Linear.  x the acute crossing angle is [0, PI / 2]
def anglePenaltyFunc(x, m):
    if x == 0: 
        return 0
    else:
        return m * (ALPHA - 1) / (math.pi / 2) * x + m

In [6]:
# Get the center and radius of a node or subgraph v
def getNodeCircle(v):
    if isinstance(v, tlp.node):
        return viewLayout[v], viewSize[v][0] / 2
    else:
        bc = tlp.computeBoundingRadius(v)
        return bc[0], bc[0].dist(bc[1])

In [7]:
# Get the penalty for overlap between a node (or a subgraph / meta-node) v and an edge e
def getNodeEdgePenalty(e, v):
    s = viewLayout[graph.source(e)]
    t = viewLayout[graph.target(e)]
    center, radius = getNodeCircle(v)
    o = crossLineSegmentAndCircle(s, t, center, radius)
    if o > 0:
        max_overlap = min(math.sqrt(dist2(s, t)), radius * 2)
        p = penaltyFunc(o, max_overlap)
        print '(meta-)node {} overlaps with edge {} (overlap area: {}, penalty: {})'.format(v, e, o, p)
        return p
    else:
        return 0

In [8]:
# Get the penalty for overlap between two nodes or graphs v1 and v2, given that nodes are represented as circles
def getNodeNodePenalty(v1, v2):  
    c1, r1 = getNodeCircle(v1)
    c2, r2 = getNodeCircle(v2)
    # print 'getNNPenalty: ', v1, c1, r1, v2, c2, r2
    o = getCircleOverlap(c1, r1, c2, r2)
    if o > 0:
        size1 = math.pi * r1 ** 2
        size2 = math.pi * r2 ** 2
        p = penaltyFunc(o, min(size1, size2))
        print '(meta-)node {} overlaps with (meta-)node {} (overlap area: {}, penalty: {})'.format(v1, v2, o, p)
        return p
    else:
        return 0

In [9]:
# Get the penalty for edge intersection
def getEdgeEdgePenalty(e1, e2):
    # print e1, graph.ends(e1), e2, graph.ends(e2)
    s1, t1 = [viewLayout[x] for x in graph.ends(e1)]
    s2, t2 = [viewLayout[x] for x in graph.ends(e2)]
    isIntersect = checkLineSegmentsIntersect(s1, t1, s2, t2)
    if isIntersect:
        a = getAngleBetweenLineSegments(s1, t1, s2, t2)
        p = anglePenaltyFunc(a, 2)
        print 'Edge {} intersects with edge {}: angle {} (deg) penalty {}'.format(e1, e2, a * 180 / math.pi, p)
        return isIntersect, p
    else:
        return False, 0

In [10]:
# Get the overall node-node penalty for the whole graph
def getGraphNodeNodePenalty(graph):
    penalty = 0
            
    for n1 in graph.getNodes():
        # leaf node x leaf node
        for n2 in graph.getNodes():
            if n1.id < n2.id:        # Make sure no dupilcate pairs of nodes are considered
                penalty += getNodeNodePenalty(n1, n2)
    
        # leaf node x subgraph
        for subGraph in graph.getDescendantGraphs():
            if not subGraph.isElement(n1):
                penalty += getNodeNodePenalty(n1, subGraph)
                    
    for sub1 in graph.getDescendantGraphs():
        for sub2 in graph.getDescendantGraphs():
            if sub1.getId() < sub2.getId() and not sub2.isDescendantGraph(sub1):   # todo check if one contains the other 
                penalty += getNodeNodePenalty(sub1, sub2)
                
    return penalty

In [11]:
def getGraphNodeEdgePenalty(graph):
    penalty = 0
    for e in graph.getEdges():
        s, t = graph.ends(e)
                
        for node in graph.getNodes():
            if node != s and node != t:
                penalty += getNodeEdgePenalty(e, node)
                
        for subGraph in graph.getDescendantGraphs():
            if not subGraph.isElement(s) and not subGraph.isElement(t):
                penalty += getNodeEdgePenalty(e, subGraph)

    return penalty

In [12]:
def getGraphEdgeEdgePenalty(graph):
    penalty = 0
    
    for e1 in graph.getEdges():
        for e2 in graph.getEdges():
            if e1.id < e2.id and graph.source(e1) not in graph.ends(e2) and graph.target(e1) not in graph.ends(e2):
                isIntersect, p = getEdgeEdgePenalty(e1, e2)
                penalty += p
    return penalty

In [13]:
# Load the test graph
graph = tlp.loadGraph('test1.tlp')
print [x for x in graph.getNodes()]
print [x for x in graph.getDescendantGraphs()]

viewLayout = graph.getLayoutProperty('viewLayout')
viewSize = graph.getSizeProperty('viewSize')

[<node 0>, <node 1>, <node 2>, <node 3>, <node 4>, <node 5>, <node 6>]
[<graph "top" (id 1) >, <graph "mid" (id 3) >, <graph "mid1" (id 2) >, <graph "bot" (id 4) >]


In [14]:
n1 = graph.nodes()[0]
n2 = graph.nodes()[1]

In [15]:
print getGraphNodeNodePenalty(graph)

(meta-)node <node 2> overlaps with (meta-)node <graph "mid1" (id 2) > (overlap area: 0.425130957168, penalty: 0.582210589848)
0.582210589848


In [16]:
e = graph.edges()[5]
g = graph.getDescendantGraph('mid')
print [viewLayout[x] for x in g.getNodes()]

[(-2.81762,6.34367,0), (-4.56109,4.11955,0), (-1.77057,4.46068,0)]


In [17]:
print getGraphNodeEdgePenalty(graph)

(meta-)node <node 1> overlaps with edge <edge 5> (overlap area: 0.252412607256, penalty: 0.452412607256)
(meta-)node <graph "mid" (id 3) > overlaps with edge <edge 5> (overlap area: 2.12977304564, penalty: 3.12630224333)
(meta-)node <graph "mid1" (id 2) > overlaps with edge <edge 5> (overlap area: 1.95803101083, penalty: 2.80607807656)
(meta-)node <graph "mid1" (id 2) > overlaps with edge <edge 7> (overlap area: 0.467792765636, penalty: 1.31583983137)
7.70063275852


In [18]:
print getGraphEdgeEdgePenalty(graph)

<edge 0> (<node 0>, <node 1>) <edge 3> (<node 5>, <node 6>)
<edge 0> (<node 0>, <node 1>) <edge 4> (<node 3>, <node 4>)
<edge 0> (<node 0>, <node 1>) <edge 5> (<node 5>, <node 3>)
Edge <edge 0> intersects with edge <edge 5>: angle 61.7439960181 (deg) penalty 0.902328959678
<edge 0> (<node 0>, <node 1>) <edge 7> (<node 2>, <node 3>)
<edge 1> (<node 1>, <node 2>) <edge 3> (<node 5>, <node 6>)
<edge 1> (<node 1>, <node 2>) <edge 4> (<node 3>, <node 4>)
<edge 1> (<node 1>, <node 2>) <edge 5> (<node 5>, <node 3>)
Edge <edge 1> intersects with edge <edge 5>: angle 73.318217886 (deg) penalty 0.69656501536
<edge 1> (<node 1>, <node 2>) <edge 6> (<node 6>, <node 0>)
<edge 2> (<node 2>, <node 0>) <edge 3> (<node 5>, <node 6>)
<edge 2> (<node 2>, <node 0>) <edge 4> (<node 3>, <node 4>)
<edge 2> (<node 2>, <node 0>) <edge 5> (<node 5>, <node 3>)
<edge 3> (<node 5>, <node 6>) <edge 4> (<node 3>, <node 4>)
<edge 3> (<node 5>, <node 6>) <edge 7> (<node 2>, <node 3>)
<edge 4> (<node 3>, <node 4>) <edg